<p style="text-align: center;"> COST-EFFICIENT DEFENCE AGAINST UNMANNED AERIAL SYSTEMS </p>

---

In [154]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [155]:
import numpy as np
import numba as nb
import matplotlib.pyplot as plt

from simulation import Simulation
from sim_utils import get_combinations, num_of_drones
from asset import Asset
from effector import Effector
from observer import Observer
from threat import Threat

from utils import read_json

In [156]:
objects = read_json("objects")
threats = objects["threats"]
combinations = get_combinations(objects)
combinations

[({'type': 'active radar',
   'value': 1000,
   'parameters': {'MC': {'a': 0.00718,
     'b': -0.184,
     'c': 1.24,
     'xmin': 2,
     'xmax': 10.5},
    'FW': {'a': 0.00267, 'b': -0.12, 'c': 1.43, 'xmin': 5, 'xmax': 20}}},
  {'type': 'jamming',
   'value': 2,
   'parameters': {'MC': {'a': 0.05, 'b': 0.5, 'c': 1.35, 'xmin': 1, 'xmax': 5},
    'FW': {'a': 0.05, 'b': 0.5, 'c': 1.35, 'xmin': 1, 'xmax': 0}}}),
 ({'type': 'active radar',
   'value': 1000,
   'parameters': {'MC': {'a': 0.00718,
     'b': -0.184,
     'c': 1.24,
     'xmin': 2,
     'xmax': 10.5},
    'FW': {'a': 0.00267, 'b': -0.12, 'c': 1.43, 'xmin': 5, 'xmax': 20}}},
  {'type': 'spoofing',
   'value': 20,
   'parameters': {'MC': {'a': 0.67,
     'b': -1.8,
     'c': 1.2,
     'xmin': 0.2,
     'xmax': 1},
    'FW': {'a': 0.67, 'b': -1.8, 'c': 1.2, 'xmin': 0.2, 'xmax': 1}}}),
 ({'type': 'active radar',
   'value': 1000,
   'parameters': {'MC': {'a': 0.00718,
     'b': -0.184,
     'c': 1.24,
     'xmin': 2,
     'xmax':

In [ ]:
def simulate_all_combinations():
    value = 1_000_000
    result_dict = {}
    for combo in combinations:
        effectors = [combo[1]]
        observers = [combo[0]]
        asset = Asset(value, effectors, observers)
        threats = [Threat(**objects["threats"][i], asset=asset) for i in range(2)]
        sim = Simulation(asset, threats)
        avg_cost, cost_std, prob = sim.simulate_n_attacks(100_000)
        
        result_dict[f"{effectors[0]['type']}-{observers[0]['type']}"] = {"avg_cost": avg_cost, "cost_std": cost_std, "prob": prob}
    
    return result_dict

In [161]:
sim_res = simulate_all_combinations()

In [162]:
sim_res

{'jamming-active radar': {'avg_cost': np.float64(2002.004),
  'cost_std': np.float64(44721.38177641635),
  'prob': 0.002},
 'spoofing-active radar': {'avg_cost': np.float64(20020.4),
  'cost_std': np.float64(140142.80000000002),
  'prob': 0.02},
 'rf-active radar': {'avg_cost': np.float64(0.0),
  'cost_std': np.float64(0.0),
  'prob': 0.0},
 'remote weapon station-active radar': {'avg_cost': np.float64(27032.4),
  'cost_std': np.float64(162277.81188517428),
  'prob': 0.027},
 'jamming-low-cost': {'avg_cost': np.float64(0.0),
  'cost_std': np.float64(0.0),
  'prob': 0.0},
 'spoofing-low-cost': {'avg_cost': np.float64(18000.378),
  'cost_std': np.float64(132953.91081045763),
  'prob': 0.018},
 'rf-low-cost': {'avg_cost': np.float64(0.0),
  'cost_std': np.float64(0.0),
  'prob': 0.0},
 'remote weapon station-low-cost': {'avg_cost': np.float64(33006.633),
  'cost_std': np.float64(178672.4083658031),
  'prob': 0.033},
 'jamming-mid-segment': {'avg_cost': np.float64(2000.404),
  'cost_std': 